In [194]:
import pandas as pd
import requests
import pickle

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [233]:
spotify_client_id = 'e0e51ec7feeb45b89b165ae1007c535d'
spotify_client_secret = 'cf444557e7ad446ebd5c2d4cc6a781cf'

client_credentials_manager = SpotifyClientCredentials(spotify_client_id, spotify_client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [234]:
def dropDictKeys(dict, keys):
    for key in keys:
        try:
            del dict[key]
        except Exception as e:
            pass
    return dict

def keepDictKeys(dict, keys):
    new_dict = {}
    if dict:
        for key in keys:
            new_dict[key] = dict[key]
        return new_dict
    return {}

In [247]:
artist_lyrics = pickle.load(open('../collection/Jhené Aiko.pkl', 'rb'))

In [248]:
%time
artist_lyrics.keys()

Wall time: 0 ns


dict_keys(['header_image_url', 'id', 'image_url', 'is_verified', 'name', 'url', 'iq', 'albums'])

In [249]:
%time
artist_lyrics['albums'][0]['tracks'][0].keys()

Wall time: 0 ns


dict_keys(['number', '_type', 'full_title', 'header_image_thumbnail_url', 'header_image_url', 'id', 'lyrics_state', 'song_art_image_thumbnail_url', 'song_art_image_url', 'stats', 'title', 'title_with_featured', 'url', 'lyrics', 'description', 'embed_content', 'featured_video', 'recording_location', 'apple_music_id', 'apple_music_player_url', 'release_date', 'release_date_for_display', 'featured_artists', 'producer_artists', 'writer_artists', 'custom_performances', 'song_relationships'])

In [250]:
%time
for artist in [artist_lyrics]:
    artist_match = sp.search(artist['name'], type='artist')
    artist_match = artist_match['artists']['items'][0]

    keep = ['followers', 'popularity', 'genres'] #'id', 'name', 
    drop = ['external_urls', 'type', 'href', 'uri', 'images']

    artist = {**artist, **keepDictKeys(artist_match, keep)}
    for i, album in enumerate(artist['albums']):

        album_match = sp.search(album['name'].split('-')[0], type='album')['albums']['items']
        if album_match:

            album_id = album_match[0]['id']
            album_match = sp.album(album_id)

            keep = ['genres', 'popularity', 'release_date'] # 'name', 'id', 'type', 'tracks'
            artist['albums'][i] = {**album, **keepDictKeys(album_match, keep)}
            
            
            for j, track in enumerate(album['tracks']):
                
                query = f"artist:{artist['name']} track:{track['title']}"
                track_match = sp.search(query, type='track')['tracks']['items']
            

                if track_match:

                    track_id = track_match[0]['id']
                    track_match = sp.track(track_id)

                    audio_features = sp.audio_features(track_id)[0]
                    audio_features = audio_features if audio_features else {}
                    drop = ['id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'type']
                    track['audio_features'] = dropDictKeys(audio_features, drop)

                    keep =['popularity', 'duration_ms'] #'id', 'name'
                    artist['albums'][i]['tracks'][j] = {**track, **keepDictKeys(track_match, keep)}

Wall time: 0 ns


In [251]:
artist['albums'][0]['tracks'][0]['audio_features']

{'danceability': 0.48,
 'energy': 0.274,
 'key': 0,
 'loudness': -17.702,
 'mode': 0,
 'speechiness': 0.316,
 'acousticness': 0.914,
 'instrumentalness': 0,
 'liveness': 0.383,
 'valence': 0.547,
 'tempo': 94.539}

In [252]:
# sp.audio_analysis(track_id)['track'].keys()

In [253]:
album['name']

'My Name Is Jhené'

In [254]:
%time
for artist in [artist_lyrics]:
    artist_match = sp.search(artist['name'], type='artist')
    artist_match = artist_match['artists']['items'][0]

    keep = ['followers', 'popularity', 'genres'] #'id', 'name', 
    drop = ['external_urls', 'type', 'href', 'uri', 'images']

    artist = {**artist, **keepDictKeys(artist_match, keep)}
    for i, album in enumerate(artist['albums']):

        album_match = sp.search(album['name'].split('-')[0], type='album')['albums']['items']
        if album_match:

            album_id = album_match[0]['id']
            album_match = sp.album(album_id)

            keep = ['genres', 'popularity', 'release_date'] # 'name', 'id', 'type', 'tracks'
            artist['albums'][i] = {**album, **keepDictKeys(album_match, keep)}
            
            track_ids = []
            for _j, track in enumerate(album['tracks']):
                
                query = f"artist:{artist['name']} track:{track['title']}"
                track_match = sp.search(query, type='track')['tracks']['items']
            
                if track_match:
                    track_id = track_match[0]['id']
                else:
                    track_id = 'None'
                track_ids.append(track_id)

            tracks = []
            fives, remainder = len(track_ids) // 5, len(track_ids) % 5
            m, n = 0, 5
            for _i in range(fives + 1):
                if track_ids[m:n]:
                    tracks = [*tracks, *sp.tracks(track_ids[m:n])['tracks']]
                    m += 5; n += 5
            tracks = [track if track else {} for track in tracks]

            audio_features = sp.audio_features(track_ids)
            audio_features = [af if af else {} for af in audio_features]

            for j, track in enumerate(album['tracks']):
                
                drop = ['id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'type']
                track['audio_features'] = dropDictKeys(audio_features[j], drop)

                keep =['popularity', 'duration_ms'] #'id', 'name'
                artist['albums'][i]['tracks'][j] = {**track, **keepDictKeys(tracks[j], keep)}

Wall time: 0 ns


In [255]:
artist['albums'][0]['tracks'][0]['audio_features']

{'danceability': 0.48,
 'energy': 0.274,
 'key': 0,
 'loudness': -17.702,
 'mode': 0,
 'speechiness': 0.316,
 'acousticness': 0.914,
 'instrumentalness': 0,
 'liveness': 0.383,
 'valence': 0.547,
 'tempo': 94.539}